In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, Point, LineString
import matplotlib.pyplot as plt
import os
import urllib

from zipfile import ZipFile
import math
%matplotlib inline

In [2]:
os.getenv("PUIDATA")

'/Users/hemingyi/PUI2018/PUIdata'

In [28]:
url = 'https://data.cityofnewyork.us/api/geospatial/uxpt-rzip?method=export&format=Shapefile'
urllib.request.urlretrieve(url, "centerline.zip")
!mv centerline.zip $PUIDATA


In [29]:
!unzip -o $PUIDATA/centerline.zip -d $PUIDATA

mv: centerline.zip: No such file or directory
Archive:  /Users/hemingyi/PUI2018/PUIdata/centerline.zip
  inflating: /Users/hemingyi/PUI2018/PUIdata/geo_export_a55fce54-f07f-4cd3-a213-255174e5c7dd.dbf  
  inflating: /Users/hemingyi/PUI2018/PUIdata/geo_export_a55fce54-f07f-4cd3-a213-255174e5c7dd.shp  
  inflating: /Users/hemingyi/PUI2018/PUIdata/geo_export_a55fce54-f07f-4cd3-a213-255174e5c7dd.shx  
  inflating: /Users/hemingyi/PUI2018/PUIdata/geo_export_a55fce54-f07f-4cd3-a213-255174e5c7dd.prj  


In [3]:
shp = gpd.GeoDataFrame.from_file(os.getenv('PUIDATA') + '/' + 'geo_export_a55fce54-f07f-4cd3-a213-255174e5c7dd.shp')

In [16]:
shp_mht = shp[shp['boroughcod'] == '1']
shp_mht = shp_mht[['bike_lane', 'geometry']]
# shp_mht.plot()

In [17]:
shp_mht.reset_index(inplace=True)

# 6 get end point and start point

In [18]:
def f(x):    
    return x['geometry'].coords
shp_mht['coords'] = shp_mht.apply(f, axis=1)

In [19]:
def whether(x):
    if len(x['coords']) > 2:
        return 0
    else:
        return 1
shp_mht['drop'] = shp_mht.apply(whether, axis=1)
shp_mht = shp_mht[shp_mht['drop'] == 1]

In [20]:
def start(x):
    return x['coords'][0]
def end(x):
    return x['coords'][1]
shp_mht['startpoint'] = shp_mht.apply(start, axis=1)
shp_mht['endpoint'] = shp_mht.apply(end, axis=1)

In [21]:
shp_mht.head()

,index,bike_lane,geometry,coords,drop,startpoint,endpoint
1,21,None,LINESTRING (-73.9626945760758 40.7607460524364...,"((-73.9626945760758, 40.76074605243648), (-73....",1,"(-73.9626945760758, 40.76074605243648)","(-73.96152208563745, 40.7602303599116)"
2,34,None,LINESTRING (-73.98391366493104 40.711303332442...,"((-73.98391366493104, 40.71130333244239), (-73...",1,"(-73.98391366493104, 40.71130333244239)","(-73.98361462167439, 40.71132564088042)"
4,58,None,LINESTRING (-73.94778083851676 40.830263780459...,"((-73.94778083851676, 40.83026378045939), (-73...",1,"(-73.94778083851676, 40.83026378045939)","(-73.94732595299794, 40.83088799495896)"
5,65,None,LINESTRING (-73.93547108195284 40.849144383074...,"((-73.93547108195284, 40.849144383074204), (-7...",1,"(-73.93547108195284, 40.849144383074204)","(-73.93496529247663, 40.849839999219384)"
6,81,None,LINESTRING (-73.93419101755205 40.814089650452...,"((-73.93419101755205, 40.81408965045272), (-73...",1,"(-73.93419101755205, 40.81408965045272)","(-73.93406888335245, 40.81409859818532)"


In [22]:
def azimuthAngle(x):
    x1 = x['startpoint'][0]
    y1 = x['startpoint'][1]
    x2 = x['endpoint'][0]
    y2 = x['endpoint'][1]
    angle = 0.0;
    dx = x2 - x1
    dy = y2 - y1
    if  x2 == x1:
        angle = math.pi / 2.0
        if  y2 == y1 :
            angle = 0.0
        elif y2 < y1 :
            angle = 3.0 * math.pi / 2.0
    elif x2 > x1 and y2 > y1:
        angle = math.atan(dx / dy)
    elif  x2 > x1 and  y2 < y1 :
        angle = math.pi / 2 + math.atan(-dy / dx)
    elif  x2 < x1 and y2 < y1 :
        angle = math.pi + math.atan(dx / dy)
    elif  x2 < x1 and y2 > y1 :
        angle = 3.0 * math.pi / 2.0 + math.atan(dy / -dx)
    return (angle * 180 / math.pi)

In [23]:
shp_mht['angle'] = shp_mht.apply(azimuthAngle, axis=1)

In [24]:
# calculate the mid point
def mid_lng(x):
    return (x['startpoint'][0] + x['endpoint'][0]) / 2
def mid_lat(x):
    return (x['startpoint'][1] + x['endpoint'][1]) / 2
shp_mht['mid_lng'] = shp_mht.apply(mid_lng, axis=1)
shp_mht['mid_lat'] = shp_mht.apply(mid_lat, axis=1)

In [25]:
del shp_mht['index']
del shp_mht['drop']
del shp_mht['startpoint']
del shp_mht['endpoint']
del shp_mht['coords']
shp_mht.head()

,bike_lane,geometry,angle,mid_lng,mid_lat
1,None,LINESTRING (-73.9626945760758 40.7607460524364...,113.741172,-73.962108,40.760488
2,None,LINESTRING (-73.98391366493104 40.711303332442...,85.733673,-73.983764,40.711314
4,None,LINESTRING (-73.94778083851676 40.830263780459...,36.082047,-73.947553,40.830576
5,None,LINESTRING (-73.93547108195284 40.849144383074...,36.021276,-73.935218,40.849492
6,None,LINESTRING (-73.93419101755205 40.814089650452...,85.809912,-73.934130,40.814094


In [26]:
# output the compact file
%mkdir Output
shp_mht.to_csv('Output/anglecalculation.csv')
shp_mht.to_file('Output/anglecalculation.shp', driver='ESRI Shapefile')

mkdir: Output: File exists
